In [4]:
include("../../src/struct_data.jl")
methods(force_func)

Cubic
LennardJones


# 4 methods for generic function "force_func":
[1] force_func(p::Cubic, value) in Main at none:1
[2] force_func(p::Cubic, i, r) in Main at /home/nicomosty/Documentos/Maestria-GitHub/CellAggregate.jl/src/forces/forces_func.jl:20
[3] force_func(p::LennardJones, value) in Main at none:1
[4] force_func(p::LennardJones, i, r) in Main at /home/nicomosty/Documentos/Maestria-GitHub/CellAggregate.jl/src/forces/forces_func.jl:26

# Forcetype

In [5]:
N = 10

cobic = Cubic(
    repeat([1.0],N) |> cu,
    repeat([2.0],N) |> cu,
    repeat([3.0],N) |> cu
)


Cubic{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}(Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], Float32[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0], Float32[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0])

In [6]:
out = zeros(size(cobic.μ₁,1)) |> cu
value =collect(0.5:0.5:size(out,1)/2) |> cu

10-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.5
 1.0
 1.5
 2.0
 2.5
 3.0
 3.5
 4.0
 4.5
 5.0

In [7]:
function add!(c,out,value)
    x = (blockIdx().x - 1) * blockDim().x + threadIdx().x 
    if x <= size(out,1)
        out[x] = force_func(c,x,value[x])
    end
    return
end

threads = 2^8
blocks = cld(size(out,1),threads)
@cuda threads=threads blocks=blocks add!(cobic,out,value)
out

10-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
   9.375
   4.0
   1.125
  -0.0
  -0.125
  -0.0
  -0.375
  -2.0
  -5.625
 -12.0

# Model

In [8]:
Model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 10000.0,
        dt    = 0.5,
        nₖₙₙ  = 50,
        nₛₐᵥₑ = 100
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input = "../../data/init/Sphere"
    )
)
dump(Model)

ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 10000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 50
    nₛₐᵥₑ: Int64 100
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"


# Aggregate

In [9]:
X = Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
Y = Float64.(readdlm("../../data/init/Sphere/10.0.xyz")[3:end,2:end]) |> cu

Complex = Nothing
Complex = Aggregate(
    [
        AggType("A", InteractionPar(Cubic(1.0,2.0,3.0), ContractilePar(0.01)),X),
        AggType("B", InteractionPar(Cubic(2.0,2.0,3.0), ContractilePar(0.02)),Y),
        AggType("C", InteractionPar(Cubic(3.0,2.0,3.0), ContractilePar(0.03)),X),
        AggType("D", InteractionPar(Cubic(4.0,2.0,3.0), ContractilePar(0.04)),Y),
    ],
    [
        AggLocation("A",[100 0   0  ]),
        AggLocation("C",[0   100 0  ]),
        AggLocation("D",[0   100 100]),
        AggLocation("B",[0   100 0  ])
    ],
    Model
)
show_aggregates(Complex)

========================= Type =======================


4-element Vector{AggType}:
 AggType("A", InteractionPar(Cubic{Float64}(1.0, 2.0, 3.0), ContractilePar(0.01)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})
 AggType("B", InteractionPar(Cubic{Float64}(2.0, 2.0, 3.0), ContractilePar(0.02)), 10.43f0, Float32[-1.5 -2.89 -8.98; 0.5 -2.89 -8.98; … ; 0.5 2.89 8.98; 2.5 2.89 8.98], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})
 AggType("C", InteractionPar(Cubic{Float64}(3.0, 2.0, 3.0), ContractilePar(0.03)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})
 AggType("D", InteractionPar(Cubic{Float64}(4.0, 2.0, 3.0), ContractilePar(0.04)), 10.43f0, Float32[-1.5 -2.89 -8.98; 0.5 -2.89 -8.98; … ; 0.5 2.89 8.98; 2.5 2.89 8.98], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})

======================   Index =======================
Index of List of Aggregates


1×6500 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  2  2  2  2  2  2  2  2  2  2  2  2

Index of Number of Aggregates


...

Contractile


ContractilePar(Float32[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01  …  0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02])

Radius


4-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 15.27
 10.43
 15.27
 10.43

------------------ Neighbors Size -------------------
idx      = (6500, 6500)
idx_red  = (21, 6500)
idx_sum  = (1, 6500)
idx_cont = (50, 6500)
------------------- Forces Size ---------------------
dX       = (6500, 3)
F        = (6500, 3)
